<a href="https://colab.research.google.com/github/ccarpenterg/LearningMXNet/blob/master/05_cifar_10_challenging_convnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CIFAR-10: A More Challenging Dataset for CNNs

In [0]:
!nvcc --version

In [0]:
!pip install mxnet-cu100

In [0]:
from __future__ import print_function

import mxnet as mx
from mxnet import nd, gluon, autograd
from mxnet.gluon import nn

import statistics

print(mx.__version__)

In [0]:
CIFAR = gluon.data.vision.CIFAR10

train_data = CIFAR(train=True)
valid_data = CIFAR(train=False)